## Importing Libraries

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 9.6MB/s eta 0:00:01
Libraries imported.


## Scrapping the Data

In [16]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [17]:
soup = BeautifulSoup(data, 'html.parser') #Parsing the data into BeautifulSoup Object
soup


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Suburbs of Mumbai - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XrKsiQpAML8AADIVVssAAABH","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Suburbs_of_Mumbai","wgTitle":"Suburbs of Mumbai","wgCurRevisionId":289157432,"wgRevisionId":289157432,"wgArticleId":8732879,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on Wikidata","Neighbourhoods in Mumbai","Suburbs by city"],"wgPageContentLanguage":"en","wgPageContentModel":"wikit

In [18]:
neighborhoodList = []

In [19]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [20]:
mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mum_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [21]:
mum_df.shape

(42, 1)

## Getting Geographical coordinates 

In [29]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, Maharashtra, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [30]:
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00426978951916, 72.85579215153318],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21198153260436, 72.83757275783374],
 [19.048530000000028, 72.93222000000003

In [31]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [32]:
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [33]:
# check the neighborhoods and the coordinates
print(mum_df.shape)
mum_df

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [34]:
#mum_df.to_csv("mum_df.csv", index=False)

In [35]:
address = 'Mumbai, Maharashtra, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, Maharashtra, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, Maharashtra, India 18.9387711, 72.8353355.


In [36]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [39]:
map_mum.save('map_mum.html')

## Using Foursquare 

In [40]:
CLIENT_ID = 'BC3LOQ3SKB3IXPKTDQJAIENREWXTR4AQ4ZJ35RW5ALLCVTHT' # your Foursquare ID
CLIENT_SECRET = 'ECMZIOMQK2WRGY0DSDKIK2TFOD4ACUSRO2BRP4LRPBWHFH5R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BC3LOQ3SKB3IXPKTDQJAIENREWXTR4AQ4ZJ35RW5ALLCVTHT
CLIENT_SECRET:ECMZIOMQK2WRGY0DSDKIK2TFOD4ACUSRO2BRP4LRPBWHFH5R


In [49]:
radius = 4000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [45]:
results[0:2] 

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b7fecb5f964a520c04430e3',
   'name': 'Worli Sea Face',
   'location': {'address': 'Worli',
    'crossStreet': 'Khan Abdul Gaffar Khan Road',
    'lat': 19.009216497075652,
    'lng': 72.8150224685669,
    'labeledLatLngs': [{'label': 'display',
      'lat': 19.009216497075652,
      'lng': 72.8150224685669}],
    'distance': 278,
    'postalCode': '400018',
    'cc': 'IN',
    'city': 'Mumbai',
    'state': 'Mahārāshtra',
    'country': 'India',
    'formattedAddress': ['Worli (Khan Abdul Gaffar Khan Road)',
     'Mumbai 400018',
     'Mahārāshtra',
     'India']},
   'categories': [{'id': '4bf58dd8d48988d165941735',
     'name': 'Scenic Lookout',
     'pluralName': 'Scenic Lookouts',
     'shortName': 'Scenic Lookout',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/sceniclookout_',
      

In [46]:
venues[0:2]

[('Andheri',
  19.118459378296492,
  72.84176321065843,
  'Merwans Cake shop',
  19.119300215885474,
  72.84541776016009,
  'Bakery'),
 ('Andheri',
  19.118459378296492,
  72.84176321065843,
  'Radha Krishna Veg Restaurant',
  19.115130144160013,
  72.84305962889206,
  'Indian Restaurant')]

In [50]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3661, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118459,72.841763,Narayan Sandwich,19.121398,72.850270,Sandwich Place


In [51]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,54,54,54,54,54,54
Baiganwadi,89,89,89,89,89,89
Bandra,100,100,100,100,100,100
Bhandup,80,80,80,80,80,80
Borivali,100,100,100,100,100,100
Charkop,100,100,100,100,100,100
Chembur,100,100,100,100,100,100
Dahisar,100,100,100,100,100,100


In [52]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 171 uniques categories.


In [59]:
"Shopping Mall" in venues_df['VenueCategory'].unique() #Checking Whether Mall is there on the list or not 

True

## Analyzing each Neighborhood by OneHot Coding 

In [62]:
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(3661, 172)


,Neighborhoods,Afghan Restaurant,Airport,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Resort,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store,Zoo
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [64]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(41, 172)


,Neighborhoods,Afghan Restaurant,Airport,Airport Service,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pizza Place,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Resort,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Water Park,Wine Bar,Women's Store,Zoo
0,Andheri,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000,0.00,0.030000,0.000000,0.030000,0.000000,0.030000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.010000,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.090000,0.000000,0.020000,0.000000,0.010000,0.020000,0.030000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.0000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.020000,0.010000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.040000,0.000000,0.130000,0.000000,0.00,0.010000,0.000000,0.00,0.020000,0.000000,0.000000,0.050000,0.010000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.030000,0.000000,0.00,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.010000,0.00,0.020000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Anushakti Nagar,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.037037,0.000000,0.00,0.000000,0.00,0.018519,0.000000,0.018519,0.000000

In [65]:
len(mum_grouped[mum_grouped["Shopping Mall"] > 0])

24

In [73]:
mum_mall = mum_grouped[["Neighborhoods","Shopping Mall"]]
mum_mall

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.012500
5,Borivali,0.000000
6,Charkop,0.020000
7,Chembur,0.020000
8,Dahisar,0.010000
9,Devipada,0.010000


## Clustering 

In [76]:
kclusters = 3

mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 1, 1, 2, 2], dtype=int32)

In [77]:
mum_merged = mum_mall.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [78]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.0000,0
1,Anushakti Nagar,0.0000,0
2,Baiganwadi,0.0000,0
3,Bandra,0.0000,0
4,Bhandup,0.0125,2


In [79]:
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() 

(41, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0000,0,19.118459,72.841763
1,Anushakti Nagar,0.0000,0,19.042830,72.927340
2,Baiganwadi,0.0000,0,19.062940,72.926630
3,Bandra,0.0000,0,19.054370,72.840170
4,Bhandup,0.0125,2,19.145560,72.948560


In [80]:
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(41, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
31,"Sion, Mumbai",0.000000,0,19.043410,72.863320
30,Shil Phata,0.000000,0,19.146580,73.040050
29,Seven Bungalows,0.000000,0,19.131342,72.816342
27,Mumbra,0.000000,0,18.940170,72.834830
25,Mira Road,0.000000,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.000000,0,18.950694,72.827268
23,Mankhurd,0.000000,0,19.048530,72.932220
17,Kalyan,0.000000,0,18.953937,72.820367
15,Jogeshwari,0.000000,0,19.137920,72.849410


In [81]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [82]:
map_clusters.save('map_clusters.html')

## Examining the Clusters 

### Cluster 0

In [83]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
31,"Sion, Mumbai",0.0,0,19.043410,72.863320
30,Shil Phata,0.0,0,19.146580,73.040050
29,Seven Bungalows,0.0,0,19.131342,72.816342
27,Mumbra,0.0,0,18.940170,72.834830
25,Mira Road,0.0,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.0,0,18.950694,72.827268
23,Mankhurd,0.0,0,19.048530,72.932220
17,Kalyan,0.0,0,18.953937,72.820367
15,Jogeshwari,0.0,0,19.137920,72.849410


In [85]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
12,Ghatkopar,0.020000,1,19.086523,72.909008
37,Vikhroli,0.021978,1,19.111090,72.927810
35,Uttan,0.020000,1,19.008040,72.844270
34,Tilak Nagar (Mumbai),0.020000,1,18.996160,72.852790
32,"Sonapur, Bhandup",0.033708,1,19.163940,72.935440
28,Pestom sagar,0.020000,1,19.070640,72.902170
11,Eastern Suburbs (Mumbai),0.030000,1,19.004270,72.855792
26,Mulund,0.030000,1,19.171830,72.955650
7,Chembur,0.020000,1,19.062180,72.902410
39,Western Suburbs (Mumbai),0.020000,1,19.197010,72.827680


In [86]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,Mahavir Nagar (Kandivali),0.010000,2,19.211982,72.837573
4,Bhandup,0.012500,2,19.145560,72.948560
21,Kurla,0.010000,2,19.064980,72.880690
18,Kandivali,0.010000,2,19.211982,72.837573
8,Dahisar,0.010000,2,19.250030,72.859070
33,Thakur village,0.010000,2,19.210200,72.875410
9,Devipada,0.010000,2,19.224690,72.866050
36,Vashi,0.012821,2,19.087426,73.011435
38,Wadala,0.010526,2,19.017200,72.858170


#### Most of the shopping malls are concentrated in the South of the Mumbai city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number to totally no shopping mall within 4KM radius in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.